### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [62]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score


# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [63]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [64]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer(stop_words='english')

In [65]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [66]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [67]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101322)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101322


In [68]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25717

In [69]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [70]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [71]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [72]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [73]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [74]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.5377831 , 0.49482167, ..., 0.        , 0.        ,
       0.        ])

In [75]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([4811, 6635, 3596, ..., 9160, 1781,  827])

In [76]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [77]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [78]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [79]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [80]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [81]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.6467991505900852

### Consigna del desafío 1

**Cada experimento realizado debe estar acompañado de una explicación o interpretación de lo observado.**

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.

**3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**NO cambiar el hiperparámetro ngram_range de los vectorizadores**.

**4**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.

**Elegir las palabras MANUALMENTE para evitar la aparición de términos poco interpretables**.


## Desafio

In [82]:
rng = np.random.default_rng(seed=42)
random_documents = rng.choice(len(newsgroups_train.data), size=5, replace=False)
print(random_documents)




[8754 4965 7404 1009 4899]


In [83]:
def get_similar_documents(idx, n = 5):
  print(f'\n\n--- DOCUMENTO ORIGINAL ---')
  print(f'DOC: {idx}')
  print(f'CLASS: {newsgroups_train.target_names[y_train[idx]]}\n\n')

  print(newsgroups_train.data[idx])

  cossim = cosine_similarity(X_train[idx], X_train)[0]

  # Obtener los índices de los más similares (de mayor a menor)
  similar_indices = np.argsort(cosine_sims)[::-1]

  # Obtener los Top (N+1) índices (el primero es el doc original mismo)
  top_indices = similar_indices[0:(n_similar + 1)]
  top_scores = cosine_sims[top_indices]

  mostsim = np.argsort(cossim)[::-1][1:n+1]

  for i, idx in enumerate(mostsim):
    print(f'\n\n--- DOCUMENTO SIMILAR({i}) ---')
    print(f'DOC: {idx}')
    print(f'CLASS: {newsgroups_train.target_names[y_train[idx]]}\n\n')

    print(newsgroups_train.data[idx])

  return mostsim,



In [84]:
def palabras_mas_importanes(idx, N):
  feature_names = tfidfvect.get_feature_names_out()

  doc_vector = X_train[idx]
  doc_scores = doc_vector.toarray().flatten()
  top_indices = np.argsort(doc_scores)[::-1]
  N = 10
  print(f"Top {N} palabras más relevantes para este documento:\n")

  for i in top_indices[:N]:
      # Si el score es 0, ya no hay más palabras relevantes
      if doc_scores[i] == 0:
          break

      print(f"Palabra: {feature_names[i]:<20} | Score TF-IDF: {doc_scores[i]:.4f}")


In [85]:
doc_0_similar = get_similar_documents(random_documents[0])



--- DOCUMENTO ORIGINAL ---
DOC: 8754
CLASS: talk.religion.misc



/(hudson)
/If someone inflicts pain on themselves, whether they enjoy it or not, they
/are hurting themselves.  They may be permanently damaging their body.

That is true.  It is also none of your business.  

Some people may also reason that by reading the bible and being a Xtian
you are permanently damaging your brain.  By your logic, it would be OK
for them to come into your home, take away your bible, and send you off
to "re-education camps" to save your mind from ruin.  Are you ready for
that?  





/(hudson)
/And why is there nothing wrong with it?  Because you say so?  Who gave you
/the authority to say that, and set the standard for morality?

Why?

Because: 
I am a living, thinking person able to make choices for myself.
I do not "need" you to show me what you think is the way; I have observed
too many errors in your thinking already to trust you to make up the
rules for me.

Because:
I set the standard for m

In [86]:

feature_names = tfidfvect.get_feature_names_out()
tfidf_matrix = X_train

top_indices = doc_0_similar
N_words = 10



import matplotlib.pyplot as plt

# --- 3. Función Auxiliar para obtener Top N Palabras ---
def get_top_n_words(doc_vector, feature_names, n=10):
    """Obtiene las n palabras con mayor score TF-IDF de un vector de documento."""
    doc_scores = doc_vector.toarray().flatten()
    top_indices = np.argsort(doc_scores)[::-1][:n]

    # Creamos un diccionario de {palabra: score}
    top_words = {feature_names[i]: doc_scores[i] for i in top_indices}
    return top_words

# --- 4. Crear el Gráfico (3 filas, 2 columnas) ---
fig, axes = plt.subplots(3, 2, figsize=(15, 18))
axes = axes.flatten() # Convertir la matriz 3x2 en un array plano de 6 ejes

print("Generando gráfico...")

for i, doc_index in enumerate(top_indices):
    ax = axes[i]

    # Obtener el vector y las top N palabras
    doc_vector = tfidf_matrix[doc_index]
    top_words = get_top_n_words(doc_vector, feature_names, N_words)

    # Obtener palabras (keys) y scores (values)
    words = list(top_words.keys())
    scores = list(top_words.values())

    # Graficar barras horizontales
    ax.barh(words, scores, color='steelblue')

    # Invertir el eje Y para que la palabra con mayor score esté arriba
    ax.invert_yaxis()

    # Títulos y etiquetas
    doc_topic = newsgroups_train.target_names[newsgroups_train.target[doc_index]]
    if i == 0:
        ax.set_title(f"DOCUMENTO ORIGINAL #{doc_index}\n(Tópico: {doc_topic})", fontweight='bold')
    else:
        sim_score = top_scores[i]
        ax.set_title(f"Similar #{i}: Doc #{doc_index} (Sim: {sim_score:.3f})\n(Tópico: {doc_topic})")

    ax.set_xlabel("Score TF-IDF")

plt.tight_layout()

print("Gráfico 'top_words_comparison.png' guardado.")



palabras_mas_importanes(random_documents[0], 10)

for idx in doc_0_similar:
  palabras_mas_importanes(idx, 10)





Top 10 palabras más relevantes para este documento:

Palabra: hudson               | Score TF-IDF: 0.3989
Palabra: people               | Score TF-IDF: 0.2321
Palabra: set                  | Score TF-IDF: 0.2139
Palabra: standard             | Score TF-IDF: 0.2048
Palabra: moral                | Score TF-IDF: 0.1989
Palabra: right                | Score TF-IDF: 0.1805
Palabra: like                 | Score TF-IDF: 0.1590
Palabra: authority            | Score TF-IDF: 0.1506
Palabra: trust                | Score TF-IDF: 0.1477
Palabra: interfere            | Score TF-IDF: 0.1368
Top 10 palabras más relevantes para este documento:

Palabra: hudson               | Score TF-IDF: 0.5286
Palabra: pain                 | Score TF-IDF: 0.2515
Palabra: lover                | Score TF-IDF: 0.2360
Palabra: awareness            | Score TF-IDF: 0.2114
Palabra: insane               | Score TF-IDF: 0.2101
Palabra: morality             | Score TF-IDF: 0.1782
Palabra: relative             | Score TF-IDF: 